# 1. Data Preparation

## Dataset Download

In [ ]:
%sh  
wget http://media.sundog-soft.com/hadoop/ml-100k/u.data -O /tmp/u.data  
wget http://media.sundog-soft.com/hadoop/ml-100k/u.item -O /tmp/u.item  
wget http://media.sundog-soft.com/hadoop/ml-100k/u.user -O /tmp/u.user  

## Upload to HDFS

In [ ]:
%sh  
hadoop fs -rm -r -f /tmp/ml-100k  
hadoop fs -mkdir /tmp/ml-100k  
hadoop fs -put /tmp/u.data /tmp/ml-100k/  
hadoop fs -put /tmp/u.item /tmp/ml-100k/  
hadoop fs -put /tmp/u.user /tmp/ml-100k/  

## Load Raw Files into RDDs

In [ ]:
%pyspark
user_rdd = sc.textFile("/tmp/ml-100k/u.user")
ratings_rdd = sc.textFile("/tmp/ml-100k/u.data")
item_rdd = sc.textFile("/tmp/ml-100k/u.item")

## Parse u.user and Create DataFrame

In [ ]:
%pyspark
from pyspark.sql import Row
# Split and convert to Row objects
user_rows = user_rdd.map(lambda line: line.split('|')) \
                    .map(lambda fields: Row(
                        user_id=int(fields[0]),
                        age=int(fields[1]),
                        gender=fields[2],
                        occupation=fields[3],
                        zip_code=fields[4]
                    ))

#  Create a DataFrame and register a temporary table
user_df = spark.createDataFrame(user_rows)
user_df.createOrReplaceTempView("users")
user_df.show(5)

+---+------+----------+-------+--------+  
|age|gender|occupation|user_id|zip_code|  
+---+------+----------+-------+--------+  
| 24|     M|technician|      1|   85711|  
| 53|     F|     other|      2|   94043|  
| 23|     M|    writer|      3|   32067|  
| 24|     M|technician|      4|   43537|  
| 33|     F|     other|      5|   15213|  
+---+------+----------+-------+--------+  
only showing top 5 rows  

## Parse u.data and Create Ratings DataFrame

In [ ]:
%pyspark
ratings_rows = ratings_rdd.map(lambda line: line.split('\t')) \
                          .map(lambda fields: Row(
                              user_id=int(fields[0]),
                              movie_id=int(fields[1]),
                              rating=int(fields[2]),
                              timestamp=int(fields[3])
                          ))


ratings_df = spark.createDataFrame(ratings_rows)
ratings_df.createOrReplaceTempView("ratings")

ratings_df.show(5)

+--------+------+---------+-------+  
|movie_id|rating|timestamp|user_id|  
+--------+------+---------+-------+  
|      50|     5|881250949|      0|  
|     172|     5|881250949|      0|  
|     133|     1|881250949|      0|  
|     242|     3|881250949|    196|  
|     302|     3|891717742|    186|  
+--------+------+---------+-------+  
only showing top 5 rows  

## Parse u.item and Create Movies DataFrame

In [ ]:
%pyspark
item_rows = item_rdd.map(lambda line: line.split('|')) \
                    .map(lambda fields: Row(
                        movie_id=int(fields[0]),
                        title=fields[1],
                        release_date=fields[2],
                        video_release=fields[3],
                        imdb_url=fields[4],
                        genre_flags=fields[5:]
                    ))


item_df = spark.createDataFrame(item_rows)
item_df.createOrReplaceTempView("movies")

item_df.show(5)

+--------------------+--------------------+--------+------------+-----------------+-------------+  
|         genre_flags|            imdb_url|movie_id|release_date|            title|video_release|  
+--------------------+--------------------+--------+------------+-----------------+-------------+  
|[0, 0, 0, 1, 1, 1...|http://us.imdb.co...|       1| 01-Jan-1995| Toy Story (1995)|             |  
|[0, 1, 1, 0, 0, 0...|http://us.imdb.co...|       2| 01-Jan-1995| GoldenEye (1995)|             |  
|[0, 0, 0, 0, 0, 0...|http://us.imdb.co...|       3| 01-Jan-1995|Four Rooms (1995)|             |  
|[0, 1, 0, 0, 0, 1...|http://us.imdb.co...|       4| 01-Jan-1995|Get Shorty (1995)|             |  
|[0, 0, 0, 0, 0, 0...|http://us.imdb.co...|       5| 01-Jan-1995|   Copycat (1995)|             |  
+--------------------+--------------------+--------+------------+-----------------+-------------+  
only showing top 5 rows  

# 2. User-Movie Analysis Tasks

## i) Calculate the average rating for each movie

In [ ]:
%sql
SELECT movie_id, AVG(rating) AS avg_rating
FROM ratings
GROUP BY movie_id
LIMIT 10

## ii) Identify the top ten movies with the highest average ratings

In [ ]:
%sql
SELECT m.title, AVG(r.rating) AS avg_rating
FROM ratings r
JOIN movies m ON r.movie_id = m.movie_id
GROUP BY m.title
ORDER BY avg_rating DESC
LIMIT 10

## iii) Find the users who have rated at least 50 movies and identify their favourite movie genres

In [ ]:
%sql
WITH active_users AS (
    SELECT user_id
    FROM ratings
    GROUP BY user_id
    HAVING COUNT(*) >= 50
),
user_favs AS (
    SELECT r.user_id, m.title, COUNT(*) AS rating_count
    FROM ratings r
    JOIN active_users a ON r.user_id = a.user_id
    JOIN movies m ON r.movie_id = m.movie_id
    GROUP BY r.user_id, m.title
)
SELECT user_id, title AS favorite_movie
FROM (
    SELECT user_id, title, rating_count,
           ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY rating_count DESC) AS rn
    FROM user_favs
) t
WHERE rn = 1
LIMIT 10

## iv) Find all the users who are less than 20 years old

In [ ]:
%sql
SELECT * FROM users
WHERE age < 20
LIMIT 10

## v) Find all the users whose occupation is “scientist” and whose age is between 30 and 40 years old

In [ ]:
%sql
SELECT * FROM users
WHERE occupation = 'scientist' AND age BETWEEN 30 AND 40
LIMIT 10